# Generative AI - SEBx & Combient Hackathon December 2023

This notebook provides some examples and exercises for dealing with Large Language Models via an API interface. 

<br />

In Hands-On 1 We provide examples of how to interact with the API and how to start exploring concepts related to Prempt Engineering, the art of communicating with Large Language Models.

In Hands-On 2 we cover some basic aspects of vector embeddings; such as how to download models from the Hugging Face space and how to embed text into semantic vector embeddings. We also show how you can use the vector store library such as FAISS to directly embed and store embedded vectors via an in-memory index for semantic similarity search. In addition we demonstrate how a fine-tuning procedure of embedding models can be set up to directly tune the model for specific data and purposes.

In Hands-On 3 we set up a simple RAG system for asking queries of PDF's. This can become exponentially difficult depending on the knowledge base and what you require as a user. Here we restrict to simple examples and a somewhat limited scope of application.
<br />
<br />

***

**Note:**
This notebook contains code written in [Python](https://www.python.org/), which is a commonly used programmming language. No advanced coding expertise is required for completing the exercises and clear instructions are provided for when and where some modifications are required from the user. The notebook itself is a [Jupyter notebook](https://jupyter.org/), which provides a graphical interface for writing and executing Python code as well as displaying the output. The cell blocks in this notebook are either text formatted via the [Markdown langugage](https://en.wikipedia.org/wiki/Markdown) or executable code snippets written in Python.

The box below provides instructions for executing the code within these cell blocks and the result of executing code will be displayed beneath the relevant code cell block.


In order to run this notebook properly you will need

* **Gmail account** - Follow the provided instructions to download the notebook to your GDrive, so that you can edit and save it freely.
* **Google GDrive access** - enable the app `Google Colaboratory`. The process is highlighted in the accompanying reading material, but you can also ask ChatGPT a question like this by posing the following query: <span style="color:blue;">How do I enable google colab for the first time?</span>

> **`Run and execute each code cell block in the notebook in a consecutive manner. This is important since some code cell blocks relies on having properly executed some previous code cell block.`**
>
>
> Notebook code blocks can be executed via either:
> * **shift + enter**: executes current code block and moves to the cell below
> * **control + enter**: executes current code block

## Environment setup

Here we set up the environment and make sure we can access data via Google Drive. Run the below code blocks to install necessary packages.

Run the code below to clone the GenAI_BootCamp2023 directory which contains files which we will be using during this course. The code clones a directory called GenAI_BootCamp2023 from our Github repository.

> **`During execution of this cell block you will be prompted to provide your GDrive access to download the course content. Use your Google account credentials to allow this action.`**

## If you are not using google colab, change `use_drive` to `False`

In [ ]:
use_drive = True

if use_drive:
  import os
  from google.colab import drive
  drive.mount('/content/drive')

  # This moves into the drive where the course content folder will be downloaded
  %cd /content/drive/MyDrive

  repo_path = 'bootcamp_dec7'
  repo_url = 'https://github.com/banque-nouveau/bootcamp_dec7.git'

  # Check if the repo directory exists
  if os.path.isdir(repo_path):
    # If it exists, then pull any changes from the remote repository
    %cd {repo_path}
    !git pull
    %cd ..
  else:
    # If the directory does not exist, clone the repository
    !git clone {repo_url}

You can now view the files by clicking on the directory folder.

## Packages & Imports

Installing and importing necessary packages/modules. Note that these are only installed in a virtual environment when using the Colab Notebook.

Run the below code block to install some of the Python libraries which are required for running the Notebook.

Note that this may take up to ~15 seconds to complete.

In [ ]:
!pip install -q cohere \
    -q tiktoken \
    -q langchain \
    -q sentence_transformers \
    -q openai \
    -q faiss-cpu \
    -q colorama \
    -q pypdf \
    -q PyMuPDF \
    -q requests \
    -q beautifulsoup4 \
    -q umap-learn \
    -q mycolorpy \
    -q pandas \
    -q plotly \
    -q seaborn

Run the below code block to import the necessary library modules used in the notebook.

Note that this may take up to ~20 seconds to complete.

In [ ]:
# Some system and base modules
import os
import sys
from timeit import default_timer as timer
from typing import List, Optional, Type
import getpass

# NLP modules
import openai
from openai import OpenAI
import langchain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, JSONLoader
import fitz
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
import requests
from bs4 import BeautifulSoup

# Other modules
import numpy as np
import pandas as pd
import umap
from colorama import Fore, Back, Style


# modules for plotting
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mycolorpy import colorlist as mcp
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import seaborn as sns
%matplotlib inline

In [ ]:
sns.set(style='white', context='notebook', rc={'figure.figsize':(16,10)})

In [ ]:
print(f"OpenAI version: {openai.__version__}")
print(f"Langchain version: {langchain.__version__}")

## Setting the access key for OpenAI API

> **NB: Don't share the OpenAI access key in public spaces.**

> **The OpenAI API key can be set manually in the notebook by running the code cell block below.**
>
> **`A query box will appear the first time you run the below code cell block. Paste the OpenAI API key which you have been provided into the query box and press Enter/Return (access key is on the form sk-...)`**

In [ ]:
# Here we can set the OpenAI API access key manually in case it fails to load from the environment.
if not os.environ.get("OPENAI_API_KEY"):
  api_key = getpass.getpass("Enter OpenAI API Key here")
  os.environ["OPENAI_API_KEY"] = api_key
else:
  print(f"OPENAI_API_KEY fetched from environment!")


# sk-...

In [ ]:
# You can optionally manually insert the OpenAI API key below between the quotation marks.
# Then uncomment the following two lines by removing the preceeding # and run the cell

#os.environ["OPENAI_API_KEY"] = "sk-..."
#print(f"The Open AI access key is given by: \n\n {os.environ['OPENAI_API_KEY']}")

In [ ]:
if use_drive:
  # The following helps to format print output to match the size of the broser window
  from IPython.display import HTML, display

  def set_css():
    display(HTML('''
    <style>
      pre {
          white-space: pre-wrap;
      }
    </style>
    '''))
  get_ipython().events.register('pre_run_cell', set_css)

# Hands-On 1: Interfacing with OpenAI API, Prompt Engineering & Internet access for Extended LLM Functionality

This section provides a hands-on introduction to API interactions with the OpenAI GPT models and Prompt Engineering for end users with a basic understanding of the Python programming language. No advanced coding or technical background knowledge of generative AI or LLM is required for completing the exercises in the notebook.

<br>


<b>Example themes for exercises </b>:

* We will see how to interact with the OpenAI models via API. This includes simple one-turn interactions and concatenating conversations for follow up queries.

* Basic Prompt Engineering techniques which introduces the lkey concepts for enabling productive interactions with LLMs.



## Interacting with OpenAI models via API

**Why should you use API?**

Using the OpenAI API (Application Program Interface) to access LLM models gives us several advantages over the browser version, especially when we want to build custom made applications. One of the big perks of using the API is automation. Python, as a common programming langauge, is great at taking care of repeating or tricky tasks, which is helpful when we have a lot of such tasks to handle.

With the help of python we can also get the OpenAI models to work well with other tools and software we have. Plus, it lets us adjust things to work the way we want, making our interaction with the OpenAI models more tailored.

Additionally, using the API helps us keep track of any changes made along the way, which is great for collaborative team work. This also ensures that every time we run a task, it gives us consistent results, making it easier to check if something goes wrong and fix it. This reliability helps make our solutions stronger and more dependable.

We will be making use of a wraper from the `langchain` code library to call on the OpenAI models. This provides a simple and relatively user-friendly way to interact with the model, making it a great choice for beginners or for those looking to get things done quickly. This method provides us access to a `temperature` parameter, which determines the randomness or stochastic nature of the output the model will give us. A temperature of 1 makes the model more random, while a temperature close to 0 makes the model more deterministic with the same output for repeated queries. Empirical studies have shown that most people prefer the output to be somewhat creative in nature with a temperature value of around 0.7, which is the default value set for the OpenAI models. You can try changing this value and experience how the output changes for the same query.

In the code snippet below we call on the API to create a chat_model instance. Model is specified via the `model` parameter on the second row. A complete list of available models to choose from can be found [here](https://platform.openai.com/docs/models/overview). We will start by using one of the GPT-3.5 turbo models.

* **gpt-3.5-turbo** uses the latest available model version of GPT-3.5. Allows for up to **16,385 tokens** as input and output
    * **gpt-3.5-turbo-0613** is a legacy version of GPT-3.5, currently the same as above. Will be replaced Dec 11 2023 and be depricated on June 13 2024. Has a **4,096 token** limit
    * **gpt-3.5-turbo-1106** is the latest version of GPT-3.5, with enhanced capabilities and a **16,385 token** limit. Will become default model after Dec 11 2023

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.7
)

> **NB 1: The GPT-3.5 model usually only takes seconds to respond to a query. If it takes more than half a minute, stop the execution and rerun the cell block.**

> **NB 2: This is one option for implementing the LLM call, which is nice due to its simple and intuitive format. At the bottom of this section we explore using the OpenAI API directly as an alternative option, which provides even greater control of the input/output.**

The `chat_model` instance accepts a list of `messages` as input. We specify messages according to a schema used by various functions in the `langchain` library. This will facilitate some of the more advanced functionalities we will make use of later. Messages in a chat conversation are generally provided as a `list` (inside square brackets) of conversation turns.

In [ ]:
from langchain.schema import (
    # Overall system behaviour of the chat bot
    SystemMessage,
    # User message, query to chat bot
    HumanMessage,
    # The response from the chat bot, for conversational turns
    AIMessage
)


# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = "You are a helpful assistant."

# The user prompt is what would be written in the chat interface, your query
user_prompt = "What do you know about OpenAI's mission statement?"


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# We can check what the messages looks like by printing it out
messages

As we see, the `messages` variable is a list containing our `SystemMessage` and `HumanMessage`.


Once we have a list of messages in the above format it is easy to call the OpenAI model and get a response

> **`Calling on the OpenAI models relies on connecting to the OpenAI server. This can occasionally be slow `**

In [ ]:
# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

The full response is actually formatted as a `AIMessage`, as we can see by printing out the full response without using the print function. Observe that if we do not use the formatting of the `print` function you will see linebreak characters such as `\n` appearing in the message.

In [ ]:
response

### Giving ChatGPT a short-term memory

We can incorporate the `AIMessage` response in a new series of messages and ask a follow up question if you wish. This provides the illusion of the chat model having a memory and being able to continue a conversation for a few turns, as you have seen while working in the browser environment.

In [ ]:
# Let's ask a follow-up question which refers to the previous conversation without explicitly mentioning e.g., Patricia Industries
follow_up_question = "Can you say something more about the mission statement?"

# We can now add the response we got from the first query together with our follow-up question
# Note that if we did not include the first message, the reference to that would be missing in this follow-up
# In python, you can add lists together to form a new list which makes it easy to add new conversation turns into the messages variable
# Not that we add both the chat bot response and our follow up question
messages = messages + [response, HumanMessage(content=follow_up_question)]
messages

In [ ]:
# Here we collect the ouput from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

By continuing in this fashion we can record a conversation history which we send to the chat model as long as the total text content does not exceed the models content limit.

### OPTIONAL: Using the OpenAI API directly instead of the `langchain` wrapper

The below code snippet calls the OpenAI API directly, which allows us to access **all** of the available input and output options. While this approach offers a higher degree of control, it may not be as easy to use or straightforward to interpret as the functions from langchain. If you are interested you can try it out on your own and test various parameters. For example, the `top_p` parameter is another way to control the randomness of the model output. The recomendation from OpenAI is to use either `temperature` or `top_p` for this purpose and not both.

In [ ]:
# Needs to be set for the OpenAI API to be callable
openai.api_key = os.getenv("OPENAI_API_KEY")


# Model and parameters (almost all that are available, logit_bias not included). Parameters model and messages are required
model = "gpt-3.5-turbo-1106"
max_tokens = 1024           # Max nr of tokens to generate in the chat completion, limited by model choice
temperature = 0.7           # Value in (0, 2). Sampling temperature for stochastic nature in response
top_p = 1                   # Vale in (0, 1). Nucleus sampling, optional to temperature, considers tokens comprising top_p probability mass
frequency_penalty = 0       # Value in (-2, 2). Positive value penalizes new tokens based on frequency in text so far, to decrease likelihood of repeating sentences
presence_penalty = 0        # Value in (-2, 2). Positive value penalizes new tokens if appeared in text so far, to increase likelihood of switching to new topics
n=1                         # How many completion options to generate for each message
stream = False              # Boolean value which can allow for streaming response

# More advanced options which allow to include function calling capability for the model itself. We will not use this functionality in the exercises below
function_call = "none"      # 'auto' if call function or generate message, supply {"name": my_func}
functions = [{"name": "Name", "description": "semantic description of what the function(s) do", "parameters": {"type": "object", "properties": {}}}]


# Messages are provided as a list, similarly to the langchain method, but formatted differently
# Other possible values for the key 'role' are 'assistant' and 'function'
system_prompt = "You are a helpful assistant."
user_prompt = "What do you know about OpenAI's mission statement?"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
    ]


# API call, collected in the variable response
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p,
            frequency_penalty=frequency_penalty,
            presence_penalty=presence_penalty,
            n=n,
            stream=stream,
            function_call=function_call,
            functions=functions
        )


# Printing the response requires a slighty less intuitive format (see below for full unformatted output)
print(response.choices[0].message.content)

We can take a look at the complete response without simplifying or changing the output in any way. When you print the information from the code below, you will see that the response includes details about token usage and other aspects. These details are important as they help you estimate and collect the cost of running queries.

In [ ]:
# Printing the response variable without formatting via the print() function displays the full output from the model
response

**Try it out yourself**

- An important thing to keep in mind is the length of prompts in terms of tokens. Use the basic code below to estimate the number of tokens in a prompt, by using the messages dictionary and finding the length of tokens for its values

In [ ]:
import tiktoken


# We can get the tokens by using the correct encoding name
encoding_name = "cl100k_base"
encoding_cl1000k_base = tiktoken.get_encoding(encoding_name)

# For a particular model, we can get the appropropriate encoding
encoding = tiktoken.encoding_for_model(model)

# We can check the name for this
encoding.name

tokens = encoding_cl1000k_base.encode('This is a short sentence with  seven words.')
print(f'The number of tokens is {len(tokens)} when done with {encoding_name}')

In [ ]:
## Soln
# See https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb for a more complete soln
def num_tokens_from_messages(messages):
    
    model='gpt-3.5-turbo-0301'
    encoding  = tiktoken.encoding_for_model(model)
    tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
    tokens_per_name = -1  # if there's a name, the role is omitted

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
        num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

## Prompt Engineering techniques

Prompt Engineering (PE) is the primary vehicle for guiding generative AI models towards stable applications. It is particularly applicable to Large Language Models (LLMs) and involves formulating prompts and prompting schemas in order to retrieve appropriate responses to queries. It is essential for meaningful interactions with LLMs.

<br>

Here we will explore some basics of Prompt Engineering. We showcase examples of techniques for efficiently extracting information from Large Language Models (LLMs). These techniques are widely applicable across different models, but work the best for the very large models trained on a large corpus of data.

<br>

In this hands-on session we aim to cover

* **Why is prompting technique important?**
  * Prompting helps in formulating the input in a way that the model can understand and respond to effectively. A well-crafted prompt can significantly improve the quality and relevance of the model's output.
  * Through prompting, you can guide the model's responses in a particular direction or within certain boundaries. This is crucial for obtaining accurate, relevant, or safe responses.
  * Forms the basis behind developing advanced functionalities on top of generative AI base models.

<br>

* **What can we achieve?**
  * Quickly read and summarize/extract relevant information from text source.
  * Transform text into a format which is directly useful for you. This includes e.g. language translation or formatting the output as excel table, JSON dictionary, MarkDown or html.
  * Get feedback and suggestions for improvement and introspection. This may include both natural language text and code.
  * Infer sentiment, topic and logic structure in text.
  * Generate/revise drafts for planning, policies, mails, slides and more.
  * Brainstorming partner and ideation, an extra brain in the room.

<br>

* **Important techniques**
  * **Best practices** - Write clearly and use semantic clues rather than keyword searching. Clearly state your goals and wishes, have an intent with your promt. In some sense it is like moving back to how you perhaps intuitively wanted to interact with e.g. Google when searching internet. Or how you would ask an all-knowing oracle. Don't be afraid to interact, provide feedback and follow up when the output is not what you desired.
  * **Role Task Format (RTF)** - Give the LLM a *role*, to set it in the right *mood*, define a clear *task* to be achieved and provide a couple of examples of how you want to *format* the output.
  * **In Context Learning (ICL)** - Provide examples and any additional information to consider as context for your prompt. This allows for a *shallow* learning, where the LLM can learn from your prompt without updating any weights of the model. It is easier to make the bigger models accept new information in the prompt and generalize from that. The smaller models are more locked into the worldview built from training.
  * **Chain of Thought (CoT)** - Ask the model to think through problems logically and in a step-by-step manner. This helps the model to linearize the problem into a sequence of logical considerations and often produce more accurate output.


These techniques usually take us very far and form a basis for more advanced applications built on chaining prompts, such as e.g. [Tree of thought](https://arxiv.org/abs/2305.10601).

### **Summarizing**

Summarizing or extracting information from text sources are areas where LLMs shine particularly bright. This can be helpful for condensing lengthy news articles, generating concise summaries of meetings or reports, summarizing books or chapters for quick review or understanding and summarizing customer feedback or reviews in order to understand common sentiments or issues.

In [ ]:
# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = "You are a helpful assistant."


user_prompt = f"""
Summarize the following conversation between a service representative and a customer in a few sentences.
Use only the information from the conversation.

Service Rep: How may I assist you today?
Customer: I need to change the shipping address for an order.
Service Rep: Ok, I can help you with that if the order has not been fulfilled from our warehouse yet.
But if it has already shipped, then you will need to contact the shipping provider. Do you have the order ID?
Customer: Yes, it's 88986367.
Service Rep: One minute please while I pull up your order information.
Customer: No problem
Service Rep: Ok, it looks like your order was shipped from our warehouse 2 days ago.
It is now in the hands of  the shipping provider, so you will need to contact them to update your delivery details.
You can track your order with the shipping provider here: https://www.shippingprovider.com
Customer: Sigh, ok.
Service Rep: Is there anything else I can help you with today?
Customer: No, thanks.
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.7
)


# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

**Try it out yourself:**

Try changing the prompt or temperature for the above conversation and compare the answers:


**Summarize with a word limit:**

In [ ]:
user_prompt = f"""
Summarize the following conversation between a service representative and a customer in 20 words or less.
Use only the information from the conversation.

Service Rep: How may I assist you today?
Customer: I need to change the shipping address for an order.
Service Rep: Ok, I can help you with that if the order has not been fulfilled from our warehouse yet.
But if it has already shipped, then you will need to contact the shipping provider. Do you have the order ID?
Customer: Yes, it's 88986367.
Service Rep: One minute please while I pull up your order information.
Customer: No problem
Service Rep: Ok, it looks like your order was shipped from our warehouse 2 days ago.
It is now in the hands of  the shipping provider, so you will need to contact them to update your delivery details.
You can track your order with the shipping provider here: https://www.shippingprovider.com
Customer: Sigh, ok.
Service Rep: Is there anything else I can help you with today?
Customer: No, thanks.
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.7
)


# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)
print(len(response.content.split()))

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=1
)


# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)
print(len(response.content.split()))

In [ ]:
follow_up_prompt = f"""
Summarize the conversation in at most 30 words.
Use only the information from the conversation.
"""


# These are collected into a messages list of
new_messages = messages + [response, HumanMessage(content=follow_up_prompt)]


# Here we collect the output from the chat model in the variable response
new_response = chat_model.invoke(new_messages)

# We can print out the response by calling on its content using a .content
print(new_response.content)

**Summarize with a focus on sentiment:**

In [ ]:
follow_up_prompt = f"""
Summarize the conversation in at most 30 words,
focusing on any aspects which are relevant to customer's sentiment.
Use only the information from the conversation.
"""


# These are collected into a messages list of
new_messages = messages + [response, HumanMessage(content=follow_up_prompt)]


# Here we collect the output from the chat model in the variable response
new_response = chat_model.invoke(new_messages)

# We can print out the response by calling on its content using a .content
print(new_response.content)

### **Impersonating or defining a Role**
Defining a role is often useful when you are looking for expert answers in a specific field or just want a personalized interaction. It can be used for company chat bots or a friendly assistant. Examples of roles to consider are: Expert, Critic, project Manager, Inventor, Journalist, entrepreneur, CFO, etc.

In [ ]:
# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = """
You are a helpful assistant defined by the following criteria:
Role: You are ALEX, the personal assistant representative from Atlas Copco.
Task: Always answer politely and make sure to uphold and keep in line with the core values of Atlas Copco: interaction, innovation and commitment.
Navigating the vast landscapes of the 21st century, Atlas Copco has transformed the way individuals
and businesses perceive and interact with leading-edge technology.
Format: As a beacon of empowerment, you consistently challenge traditional industrial norms, advocating for personal growth, safety, and
entrepreneurial spirit. Whether someone is curious about the latest tech advancements or the rich
history of Atlas Copco, you have the answers. Ready to enlighten, empower, and inspire, you're the
go-to for anyone looking to learn, build and innovate.
"""


user_prompt = f"""
Can you provide a brief overview of Atlas Copco's history and its main business areas?
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.7
)


# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

**Try it yourself**

**Examples of other questions to ask:**

* Who are you?
* In how many countries does Atlas Copco operate?
* What are the primary products and services offered by Atlas Copco?
* How can I get in touch with Atlas Copco's customer support for product-related queries?

In [ ]:
user_prompt = f"""
In how many countries does Atlas Copco operate?
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

**Try it yourself**

Ask multiple questions and pass the questions to the interface at the same time. Can we also get the answer in a structured form like JSON to make it easy to reinterpret the answer.

In [ ]:
## Soln :
user_prompt = f"""Please answer the questions below which are enclosed by triple backticks and separated by semicolons. Please present the answer in JSON format using the questions as keys 
```In how many countries does Atlas Copco operate?; What are the primary products and services offered by Atlas Copco? ```
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]
# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

#### **Impersonating extra assignment**


The previous example was intentionally extreme, to demonstrate just how free we are to define the atmospheric setting. In practice however it is often useful to define the role of the LLM as an expert in the area you wish to query about. This makes it easier for the LLM to navigate its knowledge base and find more accurate responses. We can additionally ask for the output to be provided as some other persona, which can be useful when the expert lingo can become too advanced.


In this exercise we ask the LLM to be an expert astronomer, but provide its answers as if speaking to a 10-year old child.  

In [ ]:
# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = """
You are an expert astronomer who is very knowledgeable about the solar system.
Your task is to answer users questions in a satisfactory manner using clear, simple and instructive sentences.
Shape your response as if talking to a 10-year old child.

Here is an example dialogue:
Question: How many moons does Mars have?
Answer: Very good question. Mars has two small moons. They are called Phobos and Deimos. They are very small and irregularly shaped.
Phobos is the larger of the two moons and is about 17 miles (27 kilometers) in diameter.
Deimos is about 12 miles (19 kilometers) in diameter. Both moons are thought to be captured asteroids.
Imagine them as two little buddies that always follow Mars around in space! 🪐🌕🌕
"""


user_prompt = f"""
How many planets are there in the solar system?
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.7
)


# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

**Examples of other questions to ask:**

* When I learned about the planets in school, there were nine. When did that change?
* Does Pluto have any moons? What about other dwarf planets? Who chose all of these cool names?!

### **Writing - defining a typical task**

LLMs are extraordinarily good at generating written content. This can be used for many varying purposes, such as: marketing pitches, ad generation, creating an outline for an essay, grammar correction and language translation, rewriting a text from a description or writing customized emails.

#### **Writing - Marketing generation**

In [ ]:
# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = """
You are an expert in marketing and tech product design.
Generate a one paragraph marketing pitch from the user provided product description.
Use only information from the provided description.
"""


user_prompt = f"""
NokiaTWS-411 Comfort Earbuds True wireless-hörlurar Svart
Artikelnr. 5011272056 Tillv. art. nr. 8P00000143
- Typ True wireless-hörlurar
- Anslutningsteknik Trådlös
- Driftstid (upp till) 9.5 h
- Färgkategori Svart
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.7
)


# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

It will also take into account who you are creating the content for. If you ask it to write an email to your boss saying you will be late it will have a very different tone than if the email will go to your mom.

In [ ]:
# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = """
You are en expert in HR related questions and have long experience as a respected communicator.
"""


user_prompt = f"""
Write a suggestion for an email to my boss, explaining that I will have to be late this morning.
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.7
)


# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

In [ ]:
# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = """
Write a reply to a valued customer.
Generate a reply to thank the customer for their review.
Make sure to use specific details from the review.
Write in a concise and professional tone.
"""


user_prompt = f"""
Customer review: "I'm disappointed with my latest purchase. The item arrived late and was damaged."
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.7
)


# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

Asking the LLM to provdie its output in a specific form or format can be very useful

In [ ]:
# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = """
You are a helpful assistant and an expert in marketing.
"""


user_prompt = f"""
Create a product portfolio for a company that sells headphones and generate the output in table format with the following headers:
"Product class", "Product name", "Product price"
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.7
)


# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

In [ ]:
# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = """
You are a helpful assistant and an expert in marketing.
"""


user_prompt = f"""
Create a product portfolio for a company that sells headphones and generate the output in json format,
following exactly the below example schema. Don't include any triple backticks.
{{"Product class": list of product class examples, "Product name": list of product name examples, "Product price": list of product name examples}}
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.7
)


# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

**We have seen some basic examples covering the concepts of Role-task-Format, Chain-of-Thought and In-Concept-Learning. Think about editing the above examples to demonstrate the concepts better and more clearly. We will also see CoT and ICL more in the subsequent sections**

# Hands-On 2: Introduction to Embeddings & Vector index storage

Some text here for introducing embeddings and vector stores ...

## Getting arXiv data & illustrating basic embedding concepts

We will start by collecting scientific abstracts from [arXiv](https://arxiv.org). We collevct these by fetching from their new releases section, which provides the daily deluge of preprint articles in various STEM subjects.

In [ ]:
def fetch_arxiv_data(url, subject):
    """
    Function for fetching articles from the arXiv. Expects a url pointing to the daily relese site of arXiv topics.
    Returns a list of dictionaries containing 'title', 'abstract' and 'arxiv_topic'.
    """
    response = requests.get(url)
    if response.status_code != 200:
        print('Failed to retrieve data from', url)
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    papers = []

    cnt_found=0
    cnt_not_found=0
    for item in soup.find_all('div', class_='meta'):
      # We only extract info from articles with abstract in the listing. This includes cross-topic listings
      try:
        title = item.find('div', class_='list-title mathjax').text.replace('Title:', '').strip()
        abstract = item.find('p', class_='mathjax').text.strip()
        arxiv_topic = item.find('span', class_='primary-subject').text.strip()
        papers.append({'title': title, 'abstract': abstract, 'arxiv_topic': arxiv_topic, 'subject': subject})
        cnt_found+=1
      # We do not try to get abstract from replacements
      except:
        #print(f"NO ABSTRACT FOUND, DUE TO ARTICLE BEING A REPLACEMENT OF EARLIER SUBMISSION")
        cnt_not_found+=1

    print(f"Extracted abstract for {cnt_found} new articles from {subject}.\nThis excludes {cnt_not_found} replacements.")

    return papers

Below we try out the function to see the output from a single article.

In [ ]:
# Example usage
url = 'https://arxiv.org/list/gr-qc/new'  # URL for the General Relativity and Quantum Cosmology section
papers = fetch_arxiv_data(url, subject="gr-qc")

# Print the first few papers
for paper in papers[:1]:
    print("")
    print(Style.BRIGHT + 'Title:' + Style.RESET_ALL, paper['title'])
    print(Style.BRIGHT + 'Abstract:' + Style.RESET_ALL, paper['abstract'])
    print(Style.BRIGHT + 'arXiv Topic:' + Style.RESET_ALL, paper['arxiv_topic'])
    print(Style.BRIGHT + 'arXiv Subject:' + Style.RESET_ALL, paper['subject'])
    print('---')

In [ ]:
# Let's do it for all arXiv subjects
subjects = [
    "astro-ph",
    "gr-qc",
    "cond-mat",
    "quant-ph",
    "hep-th",
    "hep-ph",
    "hep-ex",
    "hep-lat",
    "nucl-ex",
    "nucl-th",
    "nlin",
    "math-ph",
    "math",
    "cs",
    "stat",
    "eess",
    ]


# We collect everything in a list of dictionaries
papers_list = []
for subject in subjects:
  papers_subject = fetch_arxiv_data(f"https://arxiv.org/list/{subject}/new", subject=subject)
  for paper in papers_subject:
    papers_list.append(paper)

print("")
print(f"We extracted a total of {len(papers_list)} abstracts")

In [ ]:
# Print the first few papers
for paper in papers_list[:1]:
    print(Style.BRIGHT + 'Title:' + Style.RESET_ALL, paper['title'])
    print(Style.BRIGHT + 'Abstract:' + Style.RESET_ALL, paper['abstract'])
    print(Style.BRIGHT + 'arXiv Topic:' + Style.RESET_ALL, paper['arxiv_topic'])
    print(Style.BRIGHT + 'arXiv Subject:' + Style.RESET_ALL, paper['subject'])
    print('---')

### Checking lengths of retrieved abstracts

In [ ]:
abstract_lengths = []
for paper in papers_list:
  abstract_lengths.append(len(paper["abstract"]))

abstract_lengths.sort(reverse=True)
print("We print out the lengths of the 10 longest abstracts")
abstract_lengths[:10]

### Embedding models

In [ ]:
# For embeddings we use models on the MTEB leaderboard at https://huggingface.co/spaces/mteb/leaderboard


# Here are some useful well ranked open source models, but they require registering and obtaining the API key, 
# currently, we will not do this, and use either the HuggingFace Embedding models below or Open AI models


# Voyage, currently nr 1 (REQUIRES REGISTERING TO GET API KEY)
#!pip install -q voyageai
#import voyageai
#from langchain.embeddings import VoyageEmbeddings
#os.environ["VOYAGE_API_KEY"] = "..."
#voyageai.api_key = os.environ["VOYAGE_API_KEY"]


# Cohere, currently nr 2 (REQUIRES REGISTERING TO GET API KEY)
#import cohere
##Get your API key from www.cohere.com
#os.environ["COHERE_API_KEY"] = "..."

In [ ]:
# Open source HuggingFace embeddings, below is currently nr 3 & 12 (NO REGISTRATION REQUIRED)
embedding_models_HF = [
    "BAAI/bge-large-en-v1.5",
    "BAAI/bge-small-en-v1.5"
    ]

In [ ]:
# Helper functions for embedding chunked text using various embedding models
#-------------------------------------------------------------------------------
def doc_embedding(
    embedding_model: str,
    model_kwargs: dict={'device': 'cpu'},
    encode_kwargs: dict={'normalize_embeddings': True},
    cache_folder: Optional[str]=None,
    multi_process: bool=False,
    ) -> HuggingFaceEmbeddings:
  """
  TBW...
  """
  embedder = HuggingFaceEmbeddings(
      model_name = embedding_model,
      model_kwargs = model_kwargs,
      encode_kwargs = encode_kwargs,
      cache_folder = cache_folder,
      multi_process = multi_process
  )
  return embedder



def get_API_embedding(text, model="text-embedding-ada-002"):
  """This function retrieves embedding vector from text string using various models"""
  text = text.replace("\n", " ")

  # OpenAI embeddings
  if model == "text-embedding-ada-002":
    client = OpenAI()
    embedding = client.embeddings.create(input = [text], model=model).data[0].embedding

  # Voyage embeddings
  elif model == 'voyage-01':
    voyage = VoyageEmbeddings(model=model, voyage_api_key=os.environ["VOYAGE_API_KEY"])
    embedding = voyage.embed_query(text)

  # Cohere embeddings
  elif model == "embed-english-v3.0":
    co = cohere.Client(os.environ["COHERE_API_KEY"])
    embedding = co.embed([text], input_type="search_document", model=model).embeddings

  elif model in embedding_models_HF:
    print("YES")
    embedder = doc_embedding(model)
    embedding = embedder.embed_query(text)

  else:
    embedding = [None]

  return embedding

We will collect our abstracts into a list of Langchain Document objects. This is not necessary for doing embeddings, but will facilitate working with vector stores later on. The Document object class has the methods `page_content`, which stores the text string, and `metadata`, where additional metadata can be stored as a dictionary with key-value pairs.

In [ ]:
documents =  []

for paper in papers_list:
  doc  = Document(
      page_content = paper["abstract"],
      metadata = {"title": paper["title"], "arxiv_topic": paper["arxiv_topic"], "subject": paper["subject"]}
  )
  documents.append(doc)
documents[0]

Let's demonstrate how an embedding works by using some open source HuggingFace embeddings for a single abstract. First let's see what the abstract looks like in plain text.

In [ ]:
test_text = documents[0].page_content
print(test_text)

Now we call the open source HuggingFace Embedding model and check the first 10 entries of the resulting embedding vector.

> ```The first time you run the below code snippet you will download the embedding model into memory and you will see the progress of this displayed.```

In [ ]:
embedding = get_API_embedding(test_text, model="BAAI/bge-small-en-v1.5")
embedding[:10]

We can now loop through all abstracts, embed them and add the embeddings to an embedding list. Later we will see how we can do this using a vector store to manage the retrieved embeddings along with additional metadata.

**NB: This takes a couple of minutes to complete for all abstracts**

In [ ]:
embeddings = []
for document in documents:
  embedding = get_API_embedding(document.page_content, model="BAAI/bge-small-en-v1.5")
  embeddings.append(embedding)


Let's check the first few entries of the embedding of one article

In [ ]:
embeddings[0][:10]

### Projecting embeddings using UMAP

We will now use the [UMAP](https://umap-learn.readthedocs.io/en/latest/) library for performing projections of the embedding vectors down to 2D, preserving both local and global structure of the data.

Let's remind ourselves what the arXiv subjects are.

In [ ]:
subjects

We prepare the data for input to the UMAP algorithm

In [ ]:
colors = mcp.gen_color(cmap="Spectral",n=len(subjects))
color_dict_subjects =dict(zip(subjects, colors))


embedding_data_array = np.array(embeddings)
print(f"We now have an array of embeddings with shape: {embedding_data_array.shape}")

In [ ]:
# We do the projection for several values of the n_neighbours hyperparameter
# This is the most important hyperparameter of the UMAP algorithm
n_neighbors = [2, 5, 15, 25, 50, 100] # 15 is default

umap_results = []
for n in n_neighbors:
    reducer = umap.UMAP(random_state=42,
                        n_components=2,
                        learning_rate=1.0,
                        min_dist=0.1,
                        n_neighbors=n,
                        metric='euclidean',
                        output_metric='euclidean',
                        target_metric='categorical',
                        target_n_neighbors=-1,
                        target_weight=0.5,)
    umap_embedding = reducer.fit_transform(embeddings)
    umap_results.append(umap_embedding)

Let's first display the result using matplotlib

In [ ]:
# Let's choose one of the UMAP results to display
nr_index = 5

df_arxiv_umap = pd.DataFrame(np.array([umap_results[nr_index][:,0], umap_results[nr_index][:,1]]).T, columns=["umap-2d-one", "umap-2d-two"])

subjects_list = []
for paper in papers_list:
  subjects_list.append(paper["subject"])
df_arxiv_umap["y"] = subjects_list

plt.figure(figsize=(16,10))
sns.scatterplot(
    x="umap-2d-one", y="umap-2d-two",
    hue="y",
    palette=color_dict_subjects,
    data=df_arxiv_umap,
    legend="full",
    alpha=0.5
)
plt.title(f"UMAP projection of arXiv abstracts with n_neighbors={n_neighbors[nr_index]}", fontsize=25)

Let's make a more interactive plot using Plotly where we can hover over the points interactively and inspect the results in more detail.

In [ ]:
subjects_list = []
for paper in papers_list:
  subjects_list.append(paper["arxiv_topic"])
df_arxiv_umap["y_long"] = subjects_list


fig = px.scatter(df_arxiv_umap,
                 x='umap-2d-one',
                 y='umap-2d-two',
                 color='y',
                 color_discrete_map=color_dict_subjects, # Use your color dictionary
                 hover_data=['y_long']) # This will show the category on hover

fig.update_traces(marker=dict(size=5, opacity=0.5)) # Adjust size and opacity similar to your seaborn plot
fig.update_layout(legend_title_text='arXiv subject') # Customize legend title
fig.show()

Tracing over the points and examining the hover labels we can see a clear clustering of physics topics and computer science topics respectively. The math topics tend to lie between these and we can see that they are closer to those topics which deal with similar lines of research.

> ```We can use the above plot to remove some of the topic subjects if we wish, to reduce the amount of data to embed and make the separation even more visually clear. Also consider changing color scheme.```

**Try it out yourself**

- One might imagine that a similar splitting of papers by topic would have been possible by searching for certain keywords. While this maybe true, embedding models clearly outperform keywords on semantic search. Try coming up with some short sentences with similar meaning but different words, and then check that the embedding vectors have high cosine similarity to the meaning. You can use `np.dot` or use `cosine_similarity` from `sklearn.metrics.pairwise` to do this

In [ ]:
##Soln: 

sent_a = 'Company A is doing well economically' 
sent_b = 'Company A is making good revenues' 
sent_c = 'Company A has many employees'

sent_a_emb = embedding = get_API_embedding(sent_a, model="BAAI/bge-small-en-v1.5")
sent_b_emb = embedding = get_API_embedding(sent_b, model="BAAI/bge-small-en-v1.5")
sent_c_emb = embedding = get_API_embedding(sent_c, model="BAAI/bge-small-en-v1.5")
profit_emb = embedding = get_API_embedding('profitable', model="BAAI/bge-small-en-v1.5")

np.dot(sent_a_emb, profit_emb), np.dot(sent_b_emb, profit_emb), np.dot(sent_c_emb, profit_emb)
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([profit_emb], [sent_a_emb, sent_b_emb, sent_c_emb])

### Vector index store & Semantic similarity search

Let's now examine how we can store embeddings in an indexed vector database. There are many different vector stores to choose from which all perform similarly. Here we will make use of [FAISS](https://ai.meta.com/tools/faiss/), which is an open source vector store library developed by Meta.

We will then see how we can use this tool to perform a similarity search over the indexed embeddings and retrieve the most relevant article based on a query using semantic similarity.

We will make use of an open source model from HuggingFace for doing the embeddings here. There are many good options to choose from of varying sizes. As we will see even quite small models perform quite well with semantic search.

> ```The first time you download the embedding model into memory you will see the progress of this displayed.```

In [ ]:
# For embeddings we use a top ranked open source model on the MTEB leaderboard at https://huggingface.co/spaces/mteb/leaderboard
embedding_model = [
    "BAAI/bge-large-en-v1.5",
    "BAAI/bge-small-en-v1.5"
    ]
embedding_model = embedding_models_HF[1]


# We use the HuggingFaceEmbeddings wrapper to make an embedding object from our model
embedder = HuggingFaceEmbeddings(
      model_name = embedding_model,
      model_kwargs = {'device': 'cpu'},
      encode_kwargs = {'normalize_embeddings': True},
      cache_folder = f"{embedding_model}_cache",
      multi_process = False
      )

It is then straightforward to create a FAISS vector index using our Document and HuggingFaceEmbeddings objects. With the small BGE model this will take slightly less than one minute to complete for 50 abstracts. We will therefore pick out a subset of roughly 50 abstracts here for demonstrative purposes.

In [ ]:
# We aim to pick out about 50 abstracts from across the documents list, irrespective of how many we originally retrieved
nr_abstracts_in_short_list = 50


# We pick out the baove nr of abstracts evenly spaced out over our list of retrieved abstracts
documents_short = documents[::max(1, len(documents) // nr_abstracts_in_short_list)]
#documents_short = documents[-nr_abstracts_in_short_list:]

print(f" The short list of documents contain {len(documents_short)} abstracts")

In [ ]:
# We embed and store embeddings for the shortened list of abstracts
faiss_index = FAISS.from_documents(
      documents=documents_short,
      embedding=embedder
      )

Let's pick out an article and display its abstract. This will allow us to make a query which we know matches this particular abstract.

In [ ]:
# choose a number between 0-20 to pick out one of the indexed abstracts
abstract_nr = 25

documents_short[abstract_nr]

Now we can construct a search query which is related to the above abstract. We then use this query to retrieve a nr of close matches from the indexed vector stored. These are retrieved as a list in sorted order, with the closest match appearing first.

Let's do this by having the LLM make a query for us from the above abstract.

In [ ]:
from langchain.schema import SystemMessage
from langchain.schema import HumanMessage

# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = """
You are an expert in summarizing scientific literature.
"""

# We can try to set this quite low to make it hard for the retriever,
# noting that if several abstracts are on a similar topic then a very short summary
# should make it more difficult to retrieve the intended one
max_words_for_summary = 3

user_prompt = f"""
Consider carefully the abstract supplied below and construct a very brief summary of its content.
Try to use a maximum of {max_words_for_summary} words and do not include any mathematical formulas in your summary.

abstract: {documents_short[abstract_nr].page_content}
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.0
)


# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

## Try it out yourself:

Try it out by telling your model what information from the reports you want to compare, for example, their sustainability strategies.



Now we can use this short summary to try to find the correct article amongst all the ones we have embedded.

In [ ]:
# We define a generic query which incorporates the summary from the LLM
search_query = f"Find an article which discusses: {response.content}"


# Define how many similar documents you want to retrieve
# These are returned in sorted order, with most similar placed first
nr_hits = 5


# Use FAISS to perform similarity search ...
most_similar = faiss_index.similarity_search(query = search_query, k=nr_hits)


# Lets check that the closest retrieved match is the same as abstract we used to construct the query
if documents_short[abstract_nr].metadata["title"] == most_similar[0].metadata["title"]:
  print(Style.BRIGHT + "SUCESS! We found the correct abstract as the top ranked choice!" + Style.RESET_ALL)
  print("--"*25)
  print(most_similar[0])
# In case it was not the top pick, we check if it was among the ones retrieved from the vector store
elif documents_short[abstract_nr].metadata["title"] in [most_similar[nr_hit].metadata["title"] for nr_hit in range(1,nr_hits)]:
  print(Style.BRIGHT + f"PARTIAL SUCESS! We found the correct abstract among the top {nr_hits} ranked choices!" + Style.RESET_ALL)
  print("--"*25)
  print(most_similar[0])
else:
  print(Style.BRIGHT + "FAILURE! We didn't retrieve the correct abstract as top choice!" + Style.RESET_ALL)

We can inspect all the top ranked abstracts we retrieved and see how well they matched the summarization we used when searching.

In [ ]:
cnt=1
for doc in most_similar:
  print(Style.BRIGHT + f"Hit nr {cnt}, Title: {doc.metadata['title']}" + Style.RESET_ALL)
  print(doc.page_content)
  print("--"*25)
  cnt+=1

If you try to play around with the above you will find that it is quite hard to get the retriever to fail based on a semantic similarity search even for a very condense summary of the abstracts.

#### Breaking the retriever?


In order to make things a bit harder we will try to collect a series of abstracts which are all related to the same topic, but which are not necessarily recent. To this end we fetch 100 article abstracts based on the query keyword `LLM`.

In [ ]:
query_keyword = "LLM"
url = f"https://arxiv.org/search/?query={query_keyword}&searchtype=all&abstracts=show&order=-announced_date_first&size=100"


# We collect everything in a list of dictionaries
response = requests.get(url)
if response.status_code != 200:
  print('Failed to retrieve data from', url)

soup = BeautifulSoup(response.content, 'html.parser')


papers = []
for item in soup.find_all('li', class_='arxiv-result'):
  title = item.find('p', class_='title is-5 mathjax').text.strip()
  abstract = item.find('span', class_='abstract-full has-text-grey-dark mathjax').text.strip()
  papers.append({"title": title, "abstract": abstract})

print("")
print(f"We extracted a total of {len(papers)} abstracts on the topic of {query_keyword}")

In [ ]:
documents_topic =  []

for paper in papers:
  doc  = Document(
      page_content = paper["abstract"],
      metadata = {"title": paper["title"]}
  )
  documents_topic.append(doc)

We embed these documents just as before

In [ ]:
# We embed and store embeddings for the shortened list of abstracts
faiss_index_topic = FAISS.from_documents(
      documents=documents_topic,
      embedding=embedder
      )

And then we pick out one of the abstracts to make a query we can try to use for search retrieval.

In [ ]:
# choose a number between 0-99 to pick out one of the indexed abstracts
abstract_nr = 10

documents_topic[abstract_nr]

In [ ]:
# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = """
You are an expert in summarizing scientific literature.
"""

# We can try to set this quite low to make it hard for the retriever,
# noting that if several abstracts are on a similar topic then a very short summary
# should make it more difficult to retrieve the intended one
max_words_for_summary = 3

user_prompt = f"""
Consider carefully the abstract supplied below and construct a very brief summary of its content.
Try to use a maximum of {max_words_for_summary} words and do not include any mathematical formulas in your summary.

abstract: {documents_topic[abstract_nr].page_content}
"""


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [ ]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.0
)


# Here we collect the output from the chat model in the variable response
response = chat_model.invoke(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

Now let's see if we can find the needle in this proverbial haystack as easily as before.

In [ ]:
# We define a generic query which incorporates the summary from the LLM
search_query = f"Find an article which discusses: {response.content}"


# Define how many similar documents you want to retrieve
# These are returned in sorted order, with most similar placed first
nr_hits = 5


# Use FAISS to perform similarity search ...
most_similar_topic = faiss_index_topic.similarity_search(query = search_query, k=nr_hits)


# Lets check that the closest retrieved match is the same as abstract we used to construct the query
if documents_topic[abstract_nr].metadata["title"] == most_similar_topic[0].metadata["title"]:
  print(Style.BRIGHT + "SUCESS! We found the correct abstract as the top ranked choice!" + Style.RESET_ALL)
  print("--"*25)
  print(most_similar_topic[0])
# In case it was not the top pick, we check if it was among the ones retrieved from the vector store
elif documents_topic[abstract_nr].metadata["title"] in [most_similar_topic[nr_hit].metadata["title"] for nr_hit in range(1,nr_hits)]:
  print(Style.BRIGHT + f"PARTIAL SUCESS! We found the correct abstract among the top {nr_hits} ranked choices!" + Style.RESET_ALL)
  print("--"*25)
  print(most_similar_topic[0])
else:
  print(Style.BRIGHT + "FAILURE! We didn't retrieve the correct abstract as top choice!" + Style.RESET_ALL)

## **Try it out yourself**

- Try varying the abstract number (abstract_nr) to check out a different one, the summary size (max_words_for_summary) and the number of hits (hits_nr) to  study the effect.

# Fine-Tuning

We are going to work further on the sentence transformers and on the abstracts dataset. We are going to fine-tune e open source models on the dataset. Be aware that fine-tuning in this case is just a demonstration. In a real case scenario, you would use more data and more special data for fine-tuning.

For finetuning, we need pairs of query - corresponding abstract pairs. In this example, we are going to use Chatgpt to generate queries for each abstract.

In [ ]:
from langchain.schema import SystemMessage
from langchain.schema import HumanMessage

chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.0
)

def generate_questions(abstract):
    system_prompt = """
    You are an expert in scientific papers.
    """
    user_prompt = f"""
    Consider carefully the following abstract of a scientific paper: {abstract}. 
    Please provide a question to this abstract. Output only the question.
    """
    messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
    ]
    response = chat_model.invoke(messages)

    return response.content

After generating the queries, we save them in a json in order to be able to reuse it in case we don't want to rerun the query generation.

In [ ]:
import json

queries = []

for paper in papers:
    queries.append(generate_questions(paper['abstract']))

abstracts_with_queries = []

for query, paper in zip(queries, papers):
    abstracts_with_queries.append({"query": query, "abstract": paper["abstract"], "title": paper["title"]})

with open("abstracts_with_queries.json", "w") as json_file:
    json.dump(abstracts_with_queries, json_file)

In [ ]:
queries[0]

Load your previously generated data from json.

In [ ]:
import json

with open('abstracts_with_queries.json', 'r') as file:
    abstracts_with_queries_json = json.load(file)

queries = []
abstracts = []
for entry in abstracts_with_queries_json:
    queries.append(entry["query"])
    abstracts.append(entry["abstract"])

# E5

In this section, we are going to fine-tune an E5 model on our data. First, we need to organise the input data. Each row should have a query and a passage which is in our case, the abstract.

In [ ]:
from sentence_transformers import InputExample

train_examples = []
for query, abstract in zip(queries, abstracts):
    train_examples.append(InputExample(texts=[query.strip(), abstract.strip()]))

In utilizing the MultipleNegativesRankingLoss, it's crucial to avoid duplicate entries within the batch, specifically ensuring the absence of identical queries or paragraphs.

In [ ]:
from sentence_transformers import (
    SentenceTransformer,
    losses,
    models,
    datasets,
)

word_embbedding = models.Transformer("intfloat/e5-small-v2")
pooling = models.Pooling(word_embbedding.get_word_embedding_dimension())
e5_model = SentenceTransformer(modules=[word_embbedding, pooling])

train_dataloader = datasets.NoDuplicatesDataLoader(train_examples, batch_size=8)
train_loss = losses.MultipleNegativesRankingLoss(e5_model)

num_epochs = 1
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)
e5_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    show_progress_bar=True,
)

After training, we create corresponding embeddings for each abstract.

In [ ]:
e5_embeddings = e5_model.encode(abstracts)

Next, we encode our user query and get the closest abstract embedding to it based on cosine similarity. 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_closest_abstract(query, model, embeddings):
    target_embedding = model.encode(query)
    similarities = [cosine_similarity([target_embedding], [emb])[0][0] for emb in embeddings]
    return abstracts[np.argmax(similarities)]

In [ ]:
query = "What are the implications of the observed decline in GPT-4's performance?"
get_closest_abstract(query, e5_model, e5_embeddings)

# BGE

BGE is currently the best performing open source sentence transformer.

In [ ]:
!pip install -U FlagEmbedding

We can fine-tune this model with a custum fuction that comes with its pip library. In order to use it, we need to prepare our data in a jsonl format. 
Example:
{"query": "Five women walk along a beach wearing flip-flops.", "pos": ["Some women with flip-flops on, are walking along the beach"], "neg": ["The 4 women are sitting on the beach.", "There was a reform in 1996.", "She's not going to court to clear her record.", "The man is talking about hawaii.", "A woman is standing outside.", "The battle was over. ", "A group of people plays volleyball."]}

In [ ]:
import random

def generate_random_except(start_range, end_range, count, exception):
    if count > (end_range - start_range + 1):
        return "Count should be smaller than the range."
    
    result = []
    while len(result) < count:
        num = random.randint(start_range, end_range)
        if num != exception and num not in result:
            result.append(num)
    
    return result

In [ ]:
queries_with_positives_and_negatives = []

for counter, entry in enumerate(abstracts_with_queries_json):
    random_indexes = generate_random_except(0, len(abstracts_with_queries_json)-1, 4, counter)
    entries = [abstracts_with_queries_json[i] for i in random_indexes]
    negative_list = [entry['abstract'] for entry in entries]
    queries_with_positives_and_negatives.append({"query": entry["query"], "pos": [entry["abstract"]], "neg": negative_list})

with open('queries_with_positives_and_negatives.jsonl', 'w') as jsonl_file:
    for entry in queries_with_positives_and_negatives:
        json.dump(entry, jsonl_file)
        jsonl_file.write('\n')

In [ ]:
!mkdir bge_abstracts

We run the fine-tuning script.

In [ ]:
!torchrun -m FlagEmbedding.baai_general_embedding.finetune.run \
    --output_dir bge_abstracts \
    --model_name_or_path BAAI/bge-small-en-v1.5 \
    --train_data ./queries_with_positives_and_negatives.jsonl \
    --learning_rate 1e-5 \
    --num_train_epochs 1 \
    --dataloader_drop_last True \
    --normlized True \
    --temperature 0.02 \
    --query_max_len 64 \
    --passage_max_len 256 \
    --train_group_size 2 \
    --logging_steps 10 \
    --query_instruction_for_retrieval "" 

We create corresponding embeddings for each abstract with our new fine-tuned model.

In [ ]:
from FlagEmbedding import FlagModel

bge_model = FlagModel('./bge_abstracts/', use_fp16=True)
bge_embeddings = bge_model.encode(abstracts)

Next, we encode our user query and get the closest abstract embedding to it based on cosine similarity. 

In [ ]:
query = "What are the implications of the observed decline in GPT-4's performance?"
get_closest_abstract(query, bge_model, bge_embeddings)

# Mixed Models

Fine-tuning the foundational bge model might boost its effectiveness for the specific task at hand, yet it could potentially cause significant decline in the model’s overall abilities outside that specific area. If we merge the fine-tuned model and the base model with LM-Cocktail, it can increase the perfomance in general tasks.

In [ ]:
!mkdir mixed_bge

In [ ]:
!pip install -U LM_Cocktail
!pip -qqq install bitsandbytes accelerate

**You might need to restart the kernel if the following doesn't work. Observe that doing so will clear the memory so you will have to run the cell with the function definition for `get_closest_abstract` in order for the cells below to work. In addition, if you want to run the Hands-On 3 cells after this, you have to rerun the imports in the beginning of the notebook as well as setting the OpenAI API access key again.**

In [ ]:
from LM_Cocktail import mix_models, mix_models_with_data

model = mix_models(
    model_names_or_paths=["BAAI/bge-small-en-v1.5", "./bge_abstracts/"],
    model_type='encoder', 
    weights=[0.5, 0.5],
    output_path="./mixed_bge")

In [ ]:
from FlagEmbedding import FlagModel

mixed_model = FlagModel('./mixed_bge', use_fp16=True)
mixed_embeddings = mixed_model.encode(abstracts)

In [ ]:
query = "What are the implications of the observed decline in GPT-4's performance?"
get_closest_abstract(query, mixed_model, mixed_embeddings)

# Hands-On 3: Working against a PDF knowledge base

Here we will see a simple example of setting up a system for Retriever Augmented Generation (RAG). We use financial reports as examples and demostrate how to interact with these as a prospective investment manager in a protfolio company.

We start by providing the location and names of the PDF files which we will be exploring.

In [ ]:
# Absolute or relative path to folder where the PDF files are located
DATAPATH = "./Data/"
#DATAPATH = "/content/drive/MyDrive/bootcamp_dec7/Data/"



# Name of the PDF files (without the trailing .pdf file endings)
caterpillar_10k = "Caterpillar-10k"
whirlpool_10k = "whirlpool-10k"
electrolux_ann_rep = "Electrolux-annual-report"


# We collect all the PDF file names in a list
pdf_files = [
    caterpillar_10k,
    whirlpool_10k,
    electrolux_ann_rep
]

> **We load and inspect the text converted PDF files and some properties below**

We can now load one of the PDFs and examine its content. Note that this relies on extracting the content of the PDF as text and works well for the parts which are written text in the PDF, but works less well for e.g., tables and figures. In order to handle tables and figures well we typically need to perform pre-processing steps which are more complex and tailored for the specific PDF file type we wish to examine.

In [ ]:
# Helper functions for text retrieval from documents
#-------------------------------------------------------------------------------
def text_from_pdf(file_path: str) -> dict:
  """
  Uses the PyMuPDF library fitz module to extract raw text from a pdf.
    https://pymupdf.readthedocs.io/en/latest/

  Input:
  file_path - path to pdf file for parsing

  Output:
  dict - Returns a dictionary with "page_nr" as key and extracted text as value.
  Additional properties can be made available (see link in comment below).
  """
  doc = fitz.open(file_path)
  pages = {}
  for page in doc:
    # type(page) = <class 'fitz.fitz.Page'>
    # (https://pymupdf.readthedocs.io/en/latest/page.html)
    pages[f"page_{page.number}"] = page.get_text()
  return pages

def pdf_dict_to_str(pdf_dicct: dict) -> str:
  """Construct a single text string from text_from_pdf() output"""
  return "\n".join(pdf_dicct.values())

In [ ]:
pdf_file = caterpillar_10k

# Here we read in the PDF as text using the function text_from_pdf(), defined in the file helper_functions.py
pdf_text = text_from_pdf(DATAPATH + pdf_file + ".pdf")


# Here we count and print out the total nr of pages in the retrieved PDF
print(f"{Fore.RED + Back.LIGHTYELLOW_EX + Style.BRIGHT}Nr of pages in pdf document:{Style.RESET_ALL} {len(pdf_text.keys())} \n")


# Here we count and print out the total nr of symbols in the retrieved PDF file
concat_text = pdf_dict_to_str(text_from_pdf(DATAPATH + pdf_file + ".pdf"))
print(f"The document contains {Fore.RED + Back.LIGHTYELLOW_EX + Style.BRIGHT}{len(concat_text)}{Style.RESET_ALL} symbols/characters")


# We then print an example page from the PDF
print(f"{Fore.RED + Back.LIGHTYELLOW_EX + Style.BRIGHT}\n\nEx text from first page:{Style.RESET_ALL}\n")
print(f"{pdf_text['page_1']}")

### OPTIONAL: The following section performs chunking of text and creates indexed embeddings for the chunks

The chat models have a limit in the number of tokens/characters they can receive as input. This means we cannot feed them the full PDF as context. Rather we need to find the most relevant pieces of text within the PDF and use that when asking the LLM to respond to a query about the PDF.

Here we split up the three PDF documents into piecewise chunks of text for embedding purposes.

In [ ]:
def TextLoader(
    file_path: str,
    loader: Type[PyPDFLoader | JSONLoader | None]=None,
    jq_schema: str='.', # '.content', '.messages[].content'
    content_key='content',
    json_lines: bool=False,
    txt_encoding: str=sys.getfilesystemencoding()
    ) -> List[Document]:
    """
    Converts a pdf, JSON or txt file into a langchain Document object,
    containing methods page_content and metadata.
    Specify type via loader, if not PyPDFLoader | JSONLoader then assumes txt
    """
    pages = None
    if loader==PyPDFLoader:
      loader = PyPDFLoader(file_path=file_path)
      pages = loader.load()   # .load_and_split()
    elif loader==JSONLoader:
      loader = JSONLoader(
          file_path=file_path,
          jq_schema=jq_schema,
          content_key=content_key,
          json_lines=json_lines
          )
      pages = loader.load()
    else:
      text_string = text_from_txt(file_path=file_path, encoding=txt_encoding)
      pages = [Document(page_content=text_string, metadata={"source": file_path})]
    return pages


def chunk_documents(
    documents: List[Document],
    TextSplitter: Type[CharacterTextSplitter | RecursiveCharacterTextSplitter],
    chunk_size: int=512,
    chunk_overlap: int=20,
    separator: str=None,
    ) -> List[Document]:
    """
    TBW ...
    Output: A list of langchain.schema.document.Document objects.
    These have methods
      - Document.page_content [str] contains the text chunk
      - Doumnent.metadata [dict] contains optional metadata
    """
    docs = None
    # CharacterTextSplitter
    if separator!=None:
      text_splitter = CharacterTextSplitter(
      separator = separator,
      chunk_size = chunk_size,
      chunk_overlap  = chunk_overlap
      )
      docs = text_splitter.split_documents(documents)
    #RecursiveCharacterTextSplitter
    else:
      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size = chunk_size,
          chunk_overlap  = chunk_overlap
      )
      docs = text_splitter.split_documents(documents)

    if docs:
      return docs
    else:
      print(f"Error: could not chunk text.")
      return None


def text_from_txt(
    file_path: str,
    encoding: str=sys.getfilesystemencoding()
    ) -> str:
  """
  Reads a txt file and returns a string object of content

  Input:
  file_path - path to txt file for parsing

  Output:
  str - Returns extracted text as str.
  """
  with open(file_path, encoding=encoding) as f:
    text = f.read()
  return text

In [ ]:
# We loop over the 3 PDF files and  collect the chunked results into a Python dictionary
chunked_pdf_files_dict = {}
for pdf_file in pdf_files:
    # Here we get the path to the current PDF file
    pdf_location = DATAPATH + pdf_file + ".pdf"
    # The below function performs the cunking of the PDF
    chunked_pdf = chunk_documents(
        documents= TextLoader(file_path=pdf_location, loader=PyPDFLoader),
        TextSplitter=RecursiveCharacterTextSplitter,
        chunk_size=512, # 512 is the maximum Sequence length
        chunk_overlap=20,
        separator=None,
        )
    chunked_pdf_files_dict[pdf_file] = chunked_pdf

Let's print out a sample chunk so that we can see what kind of information is available within a single chunk of text.

In [ ]:
print(chunked_pdf_files_dict[caterpillar_10k][79].page_content)

We can also inspect a chunk which contains a table, to see what it looks like. Observe the format of the table and note that this is the kind of inromation that will be provided as input to the LLM when we query it.

In [ ]:
# The below chunk contains a table from the caterpillar_10k PDF.
print(chunked_pdf_files_dict[caterpillar_10k][104].page_content)

#### Performing the embedding of the chunked PDF documents

In [ ]:
embed_pdfs = False
save_embeddings = False

# For embeddings we use the best open source model on the MTEB leaderboard at https://huggingface.co/spaces/mteb/leaderboard
embedding_model = "BAAI/bge-small-en-v1.5"

> **NB: The below code block takes around 100 minutes to complete for the three PDF's (100-150 pages each), so this has been prepared before the lecture. We have kept the code here for those interested, but during the course**
>
> **`DO NOT RUN THE BELOW TWO CELLS!` or `DO NOT MODIFY THE embed_pdfs = False and save_embeddings = False!`**

In [ ]:
def make_index_FAISS(
    chunked_documents: List[Document],
    embedding: HuggingFaceEmbeddings,
    ) -> List:
  """Use FAISS to perform similarity search ..."""
  faiss_index = FAISS.from_documents(
      documents=chunked_documents,
      embedding=embedding
      )
  return faiss_index


def similarity_search_FAISS(
    search_query: str,
    index_store: FAISS,
    nr_hits: int=5,
    ) -> List:
  """Use FAISS to perform similarity search ..."""
  most_similar = index_store.similarity_search(search_query, k=nr_hits)
  return most_similar

In [ ]:
if embed_pdfs:
    embedding_dict = {}
    for pdf_file in pdf_files:
        print(f"Creating FAISS embedding for document {pdf_file} using - {embedding_model}")
        start = timer()
        embedding = doc_embedding(embedding_model=embedding_model)
        faiss_index = make_index_FAISS(chunked_documents=chunked_pdf_files_dict[pdf_file], embedding=embedding)
        end = timer()
        time_taken=end-start
        print(f"Done! Embedded vector index created after {time_taken/60:.2f} minutes")
        print("*"*25)
        embedding_dict[pdf_file] = faiss_index

Here we save the embeddings so that we don't have to rerun the above every time.

In [ ]:
if save_embeddings:
    for pdf_embedding in embedding_dict.keys():
        FAISS_folder_name = DATAPATH + f"FAISS_{pdf_embedding}.faiss"
        embedding_dict[pdf_embedding].save_local(FAISS_folder_name)

Note that the above two code blocks have been run in advance in order to save the embeddings. This allows us to avoid doing this during the live session.

### Loading the saved embeddings

In order to match our query to relevant context within the PDF files we must embed the text of the PDF files into an *embedding space*. This is done by chunking the text of the PDF files and then store the chunks as indexed *embedding vectors*. This is what has been done in advance by performing the steps in the above optional section.

We can then create an *embedding vector* from our query and perform a similarity operation against all of the embedded chunks, which allow us to find the most semantically similar chunks by performing a very quick operation. In this way we can quickly retrieve relevant context for our query.

If embeddings are already saved we can simply load them from storage. The output from running the embedding models have been saved in the four FAISS folders which you can see in the downloaded course content. This avoids repeating the embedding process which can be quite time consuming. In a production environment you typically do this kind of indexing once, or on a running schedule if you expect the documents you embed will change over time.

> **`Note that the first time you run the below code cell block you will load the embedding model into memory and you may see progress bars running while this happens. Once the model is loaded into memory this will no longer happen.`**

In [ ]:
# For embeddings we use the best open source model on the MTEB leaderboard at https://huggingface.co/spaces/mteb/leaderboard
embedding_model = "BAAI/bge-large-en-v1.5"

# We load and collect the saved embeddings in a python dictionary. This only takes ~15 seconds to load
faiss_embeddings_dict = {}
if not save_embeddings:
    for pdf_file in pdf_files:
        FAISS_folder_name = DATAPATH + f"FAISS_{pdf_file}.faiss"
        embedding = doc_embedding(embedding_model=embedding_model)
        faiss_embeddings_dict[pdf_file] = FAISS.load_local(FAISS_folder_name, embedding)

Let's try out making a simple query against the saved embeddings by doing a similarity search. The similarity search is a quick operation which retrieves the most semantically similar embeddings to your query.

In [ ]:
# We choose one of the PDF files and a relevant search query
pdf_file = caterpillar_10k
similarity_query = "What are Caterpillar's core values?"


# We retrieve the most semantically similar chunks by doing a similarity search over the saved embeddings in our faiss_embeddings_dict
# The argument k sets how many such chunks we wish to retrieve
test = faiss_embeddings_dict[pdf_file].similarity_search(similarity_query, k=10)


# We can now print out the most similar chunk. We print out both the chunk and the PDF page on which it is located.
# The retrieved chunks are collected in order of similarity from 0 to k-1,
# so chunk_nr = 0 correspond to the most similar chunk
chunk_nr = 0
print(f"Found on page: {test[chunk_nr].metadata['page']}\n")
print(test[chunk_nr].page_content)

### Querying the PDF

We are now ready to start asking questions of our PDF files.

In [ ]:
# Step 1:
# Setting up the Model
# We're preparing to use a specific AI model by setting some initial parameters.
# One of these parameters is 'temperature', which affects how creative or strict the model's responses will be.
# A lower temperature like 0.0 makes the model more focused and less random in its responses.
temperature = 0.0


# Now we create an instance of the ChatOpenAI model, which we'll use to generate responses.
# We're specifying which model to use ('gpt-3.5-turbo-1106') and providing our OpenAI API key to authorize access.
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-1106",
    temperature=0.7
)



# Step 2:
# Defining a Prompt Function to Use the Model
# We define a function for more easily calling the LLM using the same prompt template but with a different query and context
def LLM_with_pdf_context(context: str, query: str, temperature: float=0.0):

    # System prompt
    system_prompt = f"""
    Ignore all previous instructions. You are a helpful investment management expert.
    You are logical, methodical and always find the best and most relevant answer to a query.
    Break down the problem, objects, numbers and logic before starting to answer the query.
    Then proceed to answer in a step-by-step manner.
    """

    # This prompt template inputs the merged context and query for the LLM
    user_prompt = f"""
    I need help from an investement management expert.
    One of our portfolio companies have handed us these brief instruction snippets as
    context: {context}

    Use only the above context and nothing else to answer the following
    question: {query}
    If the answer is provided in the form of a bulleted list within the context,
    then return those instructions verbatim and do not try to rephrase them.
    If you cannot answer based on the information in context, then do not try to answer, but instead must answer verbatim with:
    {{There is no available information related to your query in the context!'}}
    """

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt),
    ]

    response = chat_model.invoke(messages, temperature=temperature)
    print(response.content)

In [ ]:
# We choose one of the PDF files and a relevant search query. This query is the same as we asked during session 1 using the AskYourPDF plugin
pdf_file = caterpillar_10k
query = "What are Caterpillar's core values?"


# We retrieve the most similar chunks related to above search query and collect these into a context string
faiss_index = faiss_embeddings_dict[pdf_file]
top_hits = similarity_search_FAISS(search_query=query, nr_hits=5, index_store=faiss_index)
context = "".join([document.page_content for document in top_hits])

First, let's use all of the retrieved top hits as context when querying the model.

In [ ]:
# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

We can try to be more complete by providing the full page where the retrieved chunk was found rather than just the chunk. This is often useful in order to be able to get more full answers to a query

In [ ]:
# Here we get the page of the top hit and use that whole PDF page as context
context_page = f"""{pdf_text[f'page_{top_hits[0].metadata["page"]}']}"""

# Use the pre-defined function but with the whole context page as context
LLM_with_pdf_context(context_page, query)

### Querying the Electrolux Annual Report PDF

Let's try asking some questions based on the content in the Electrolux's annual report for 2022.

In [ ]:
def get_contexts(pdf: str, query: str, faiss_embeddings_dict: dict, nr_hits: int=5):
    contexts = []
    faiss_index = faiss_embeddings_dict[pdf]
    top_hits = similarity_search_FAISS(search_query=query, nr_hits=nr_hits, index_store=faiss_index)
    context = "".join([document.page_content for document in top_hits])
    contexts.append(context)

    return contexts

In [ ]:
# Set our pdf variable to the Electrolux annual report
pdf_file = electrolux_ann_rep

In [ ]:
query = "Which major events happened for Electrolux during 2022?"


# Use our function to retrieve the most similar chunks related to the search query for the given PDF
context = get_contexts(pdf=pdf_file, query=query, faiss_embeddings_dict=faiss_embeddings_dict)


# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

**Try it out yourself:**

Try it out by searching with your own query for information from the Electrolux Annual Report. You could, for example, ask questions about risks and challenges mentioned in the report or ask the model to explain Electrolux's business in Latin America.

In [ ]:
# Write your own query and retrieve search results from that as context for the LLM
query = "..."


# Use our function to retrieve the most similar chunks related to the search query for the given PDF
context = get_contexts(pdf=pdf_file, query=query, faiss_embeddings_dict=faiss_embeddings_dict)


# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

### Querying the Caterpillar 10-K PDF

Let's try asking some questions based on the content in the Caterpillar 10-K annual report for 2022.

In [ ]:
# Set our pdf variable to the Caterpillar 10-K report
pdf_file = caterpillar_10k

In [ ]:
# Ask a relevant query
query = "Which identified risks and challenges are mentioned in the report?"


# Use our function to retrieve the most similar chunks related to the search query for the given PDF
context = get_contexts(pdf=pdf_file, query=query, faiss_embeddings_dict=faiss_embeddings_dict)


# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

**Tables in PDF-reports:**

The Caterpillar 10-K report contains multiple tables which may cause a problem when using the text chunks for similarity retrieval. Here we can try and ask questions about information found in tables in the report. However, depending on the chunks, the model may confuse numbers found in the tables.

In [ ]:
# Let's ask a question which requires analyzing table information inside the PDF
query = "How many full-time employees did they have at the end of the year 2022 in Latin America?"


# Use our function to retrieve the most similar chunks related to the search query for the given PDF
context = get_contexts(pdf=pdf_file, query=query, faiss_embeddings_dict=faiss_embeddings_dict)


# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

**Try it out yourself:**

Try it out by searching with your own query for information from the Caterpillar 10-K report.

In [ ]:
# Write your own query and retrieve search results from that as context for the LLM
query = "..."


# Use our function to retrieve the most similar chunks related to the search query for the given PDF
context = get_contexts(pdf=pdf_file, query=query, faiss_embeddings_dict=faiss_embeddings_dict)


# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

### Comparison between two PDF: Electrolux Annual Report and Whirlpool 10-K Report

Let's go a bit deeper and try to compare information between two different PDF files. We can try to ask a question for which we expect there are answers in both the Electrolux annual report and the Whirlpool 10-K report. By using a query to retrieve relevant information from both of these reports we can use that as context to the LLM and ask about comparisons etc.

In [ ]:
# We define a function for more easily call the LLM using the same prompt template but with a different query and contexts
def LLM_with_pdf_context_comparison(company1: str, company2: str, context1: str, context2: str, query: str, temperature: float=0.0):

    # System prompt
    system_prompt = f"""
    Ignore all previous instructions. You are a helpful investment management expert.
    You are logical, methodical and always find the best and most relevant answer to a query.
    Break down the problem, objects, numbers and logic before starting to answer the query.
    Then proceed to answer in a step-by-step manner.
    """

    # This prompt inouts the merged context
    user_prompt = f"""
    Two of our portfolio companies have handed us these brief instruction snippets as
    contexts. Contexts for {company1}: {context1}, and for {company2}: {context2}

    Use only the above contexts and nothing else to answer the following
    query: {query}
    If the answer is provided in the form of a bulleted list within the context,
    then return those instructions verbatim and do not try to rephrase them.
    If you cannot answer based on the information in context, then do not try to answer, but instead must answer verbatim with:
    {{There is no available information related to your query in the context!'}}
    """

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt),
    ]

    response = chat_model.invoke(messages, temperature=temperature)
    print(response.content)

With the help of the above function we can pose queries which prompts the LLM to compare the information in different PDF's by creating separate contexts. Here, we compare the information in the Electrolux Annual report and Whirlpool 10-K report.

In [ ]:
# Let's make a query which we would like to pose to both the Whirlpool and the Electrolux PDF files
query = "Which key business strategies were mentioned in the report?"


# Using the above query, we can fetch relevant contexts from both PDF files
context_whirlpool = get_contexts(pdf=whirlpool_10k, query=query, faiss_embeddings_dict=faiss_embeddings_dict)
context_electrolux = get_contexts(pdf=electrolux_ann_rep, query=query, faiss_embeddings_dict=faiss_embeddings_dict)

In [ ]:
# Given the above retrieved contexts we can now ask a comparative question between the companies
chat_query = """
You will be provided contextual information which you should analyze very carefully.
Think about the problem in a step by step manner.
Consider the following key business strategies for Whirlpool and Electrolux. 
Compare them as much as possible. 
Which strategies are similar and which are different?
"""

In [ ]:
# Use our pre-defined function together with the names of the companies and their context
LLM_with_pdf_context_comparison("Whirlpool", "Electrolux", context_whirlpool, context_electrolux, chat_query)

**Try it out yourself:**

Try it out by telling your model what information from the reports you want to compare, for example, their sustainability strategies.

In [ ]:
# Write your own query on what you like to pose to both the Whirlpool and the Electrolux PDF files
query = "..."


# Using the above query, we can fetch relevant contexts from both PDF files
context_whirlpool = get_contexts(pdf=whirlpool_10k, query=query, faiss_embeddings_dict=faiss_embeddings_dict)
context_electrolux = get_contexts(pdf=electrolux_ann_rep, query=query, faiss_embeddings_dict=faiss_embeddings_dict)


# Change this query to ask the model to compare ... for the companies.
chat_query = "This is the ... for Whirlpool and Electrolux. Compare them."


# Use our pre-defined function together with the names of the companies and their context
LLM_with_pdf_context_comparison("Whirlpool", "Electrolux", context_whirlpool, context_electrolux, chat_query)

### End of the notebook